<a href="https://colab.research.google.com/github/SatishSurya1/diabetes-prediction/blob/main/cifar10CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **image classification using CNN**

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [ ]:
batch_size = 4

train_ds= torchvision.datasets.CIFAR10(root = '/.data', train =True, download=True, transform =transform)
test_ds = torchvision.datasets.CIFAR10(root= '/.data', train= False,download= True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(test_ds, batch_size = batch_size, shuffle = False)

100%|██████████| 170498071/170498071 [00:02<00:00, 68906591.85it/s]


Extracting /.data/cifar-10-python.tar.gz to /.data
Files already downloaded and verified


In [ ]:
print(train_ds.classes)
print(test_ds.classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [ ]:
 classes = ('airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck')

In [ ]:
from PIL import Image
img_index = 0
image, labels = train_ds[img_index]
image_tensor = image.squeeze()
image_pil = transforms.ToPILImage()(image_tensor)

img_size = image_tensor.shape[1:]
num_channels = image_tensor.shape[0]
print(img_size)
image_pil.show()


torch.Size([32, 32])


In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fcl1 = nn.Linear(16*5*5, 120)
    self.fcl2 = nn.Linear(120, 84)
    self.fcl3 = nn.Linear(84,10)

  #forward pass
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fcl1(x))
    x = F.relu(self.fcl2(x))
    x = self.fcl3(x)
    return x

model = ConvNet().to(device)

In [ ]:
learning_rate = 0.001
loss  = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [ ]:
num_epoch = 5

n_total_steps = len(train_loader)
for epoch in range(num_epoch):
  for i, (images,labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    output = model(images)
    l = loss(output, labels)
    #print(output)
    #print('-----------------')
   # print(labels)

    #backward pass

    optimizer.zero_grad()
    l.backward()
    optimizer.step()

    if (i+1) % 5000 == 0:
            print(f'Epoch[{epoch+1}/{num_epoch}], Step[{i+1}/{n_total_steps}], Loss:{l.item():.4f}')





Epoch[1/5], Step[5000/12500], Loss:0.9559
Epoch[1/5], Step[10000/12500], Loss:0.4587
Epoch[2/5], Step[5000/12500], Loss:1.3330
Epoch[2/5], Step[10000/12500], Loss:0.5447
Epoch[3/5], Step[5000/12500], Loss:1.1382
Epoch[3/5], Step[10000/12500], Loss:1.2918
Epoch[4/5], Step[5000/12500], Loss:1.9000
Epoch[4/5], Step[10000/12500], Loss:1.2072
Epoch[5/5], Step[5000/12500], Loss:1.1119
Epoch[5/5], Step[10000/12500], Loss:0.6219


In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of the network: 56.61 %
Accuracy of airplane: 53.6 %
Accuracy of automobile: 79.4 %
Accuracy of bird: 47.0 %
Accuracy of cat: 22.1 %
Accuracy of deer: 52.1 %
Accuracy of dog: 44.2 %
Accuracy of frog: 65.9 %
Accuracy of horse: 72.6 %
Accuracy of ship: 66.6 %
Accuracy of truck: 62.6 %


TypeError: ignored